In [1]:
!pip install --upgrade pip datashader geoviews hvplot dask

In [2]:
import hvplot.xarray 
import s3fs
import xarray as xr
import numpy as np

In [3]:
import sys
sys.path.append("./s3tree.py")  # folder that contains s3tree.py
from s3tree import print_variables, show_tree, show_tree_for_variable

bucket = "reflective-persistent-prod"
group  = "gauss"

In [4]:
# List variables with sizes
print_variables(bucket, group, show_sizes=True)

 1. ICEFRAC              614.5MB
 2. PRECC                1.0GB
 3. PRECL                1.3GB
 4. PRECT                61.8GB
 5. PRECT_above_10       4.4GB
 6. PRECT_above_20       4.4GB
 7. QFLX                 2.1GB
 8. TREFHT               45.2GB
 9. TREFHTMN             2.2GB
10. TREFHTMX             2.2GB
11. TREFHT_above_35      4.4GB
12. TREFHT_above_40      4.4GB
13. TREFHT_below_0       4.4GB


In [5]:
# Entire tree
show_tree(bucket, group)

|_ gauss/  (138.7GB total)
  |_ ICEFRAC/  (614.5MB total)
    |_ historical/  (190.8MB total)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.001.cam.h0.ICEFRAC.185001-189912.nc  (19.4MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.001.cam.h0.ICEFRAC.190001-194512.nc  (17.8MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.001.cam.h0.ICEFRAC.194601-198512.nc  (15.6MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.001.cam.h0.ICEFRAC.198601-201412.nc  (11.0MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002.cam.h0.ICEFRAC.185001-189912.nc  (19.4MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002.cam.h0.ICEFRAC.190001-194912.nc  (19.3MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002.cam.h0.ICEFRAC.195001-199912.nc  (19.2MB)
      |_ b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002.cam.h0.ICEFRAC.200001-201412.nc  (5.7MB)
      |_ b.e21.BWmaHIST.f09

In [6]:
# Only one variable
show_tree_for_variable(bucket, group, "TREFHT")

|_ gauss/TREFHT/  (45.2GB total)
  |_ gauss/  (45.2GB total)
    |_ TREFHT/  (45.2GB total)
      |_ historical/  (22.0GB total)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.18500101-18591231.nc  (442.0MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.18600101-18691231.nc  (442.0MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.18700101-18791231.nc  (441.9MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.18800101-18891231.nc  (441.9MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.18900101-18991231.nc  (441.9MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.19000101-19091231.nc  (442.0MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.19100101-19191231.nc  (441.9MB)
        |_ b.e21.BWHIST.f09_g17.CMIP6-historical-WACCM.001.cam.h1.TREFHT.19200101-19291231.nc  (441.7MB)
        |_ b.e21.BWHIST.f09_g17

# Reading all netCDF files together

In [7]:
variable_name = "TREFHT"
scenario = "historical"
partial_filename = "b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002.cam.h0.TREFHT"
fs = s3fs.S3FileSystem(anon=False)
keys = fs.glob(f"{bucket}/{group}/{variable_name}/{scenario}/{partial_filename}*.nc")
urls = [f"simplecache::s3://{k}" for k in keys]

ds_all = xr.open_mfdataset(
    urls,
    engine="h5netcdf",
    combine="by_coords",           # or "nested" + concat_dim="time"
    parallel=True,
    backend_kwargs={
        "storage_options": {
            "s3": {"anon": False},
            "simplecache": {"cache_storage": "/tmp/s3cache"},
        }
    },
)
ds_all

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'argo' loading failed:
cannot import name 'collections_to_dsk' from 'dask.base' (/srv/conda/envs/notebook/lib/python3.12/site-packages/dask/base.py)
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


<xarray.Dataset> Size: 446MB
Dimensions:       (time: 1980, zlon: 1, nbnd: 2, lat: 192, lev: 70, ilev: 71,
                   lon: 288)
Coordinates:
  * lat           (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * zlon          (zlon) float64 8B 0.0
  * lon           (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * lev           (lev) float64 560B 5.96e-06 9.827e-06 1.62e-05 ... 976.3 992.6
  * ilev          (ilev) float64 568B 4.5e-06 7.42e-06 1.223e-05 ... 985.1 1e+03
  * time          (time) object 16kB 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/27)
    zlon_bnds     (time, zlon, nbnd) float64 32kB dask.array<chunksize=(600, 1, 2), meta=np.ndarray>
    gw            (time, lat) float64 3MB dask.array<chunksize=(600, 192), meta=np.ndarray>
    hyam          (time, lev) float64 1MB dask.array<chunksize=(600, 70), meta=np.ndarray>
    hybm          (time, lev) float64 1MB dask.array<chunksize=(600, 70), meta=np.ndarray>
    P0            (time) float64 16kB 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    hyai          (time, ilev) float64 1MB dask.array<chunksize=(600, 71), meta=np.ndarray>
    ...            ...
    n2ovmr        (time) float64 16kB dask.array<chunksize=(512,), meta=np.ndarray>
    f11vmr        (time) float64 16kB dask.array<chunksize=(512,), meta=np.ndarray>
    f12vmr        (time) float64 16kB dask.array<chunksize=(512,), meta=np.ndarray>
    sol_tsi       (time) float64 16kB dask.array<chunksize=(512,), meta=np.ndarray>
    nsteph        (time) float64 16kB dask.array<chunksize=(600,), meta=np.ndarray>
    TREFHT        (time, lat, lon) float32 438MB dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg...
    logname:           mmills
    host:              cheyenne1
    initial_file:      b.e21.BWma1850.f09_g17.release-cesm2.1.3.c20200918.cam...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [8]:
temp_yearly = ds_all.groupby("time.year").mean(dim="time").isel(year=slice(None, -1))["TREFHT"]
temp_yearly

<xarray.DataArray 'TREFHT' (year: 165, lat: 192, lon: 288)> Size: 36MB
dask.array<getitem, shape=(165, 192, 288), dtype=float32, chunksize=(1, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1kB 1850 1851 1852 1853 1854 ... 2011 2012 2013 2014
Attributes:
    units:         K
    long_name:     Reference height temperature
    cell_methods:  time: mean

# Reading individual files

In [9]:
variable_name = "TREFHT"
scenario = "historical"
SAI = ""
desired_file = "b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg.002.cam.h0.TREFHT.195001-199912.nc"

url = f"simplecache::s3://{bucket}/{group}/{variable_name}/{scenario}/{desired_file}"

# simplecache downloads once to a local temp file for fast random access
ds = xr.open_dataset(
    url,
    engine="h5netcdf",        # or "netcdf4" if you prefer that backend
)
ds

<xarray.Dataset> Size: 133MB
Dimensions:       (lat: 192, lon: 288, zlon: 1, nbnd: 2, lev: 70, ilev: 71,
                   time: 600)
Coordinates:
  * lat           (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon           (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
  * zlon          (zlon) float64 8B 0.0
  * lev           (lev) float64 560B 5.96e-06 9.827e-06 1.62e-05 ... 976.3 992.6
  * ilev          (ilev) float64 568B 4.5e-06 7.42e-06 1.223e-05 ... 985.1 1e+03
  * time          (time) object 5kB 1950-02-01 00:00:00 ... 2000-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/27)
    gw            (lat) float64 2kB ...
    zlon_bnds     (zlon, nbnd) float64 16B ...
    hyam          (lev) float64 560B ...
    hybm          (lev) float64 560B ...
    P0            float64 8B ...
    hyai          (ilev) float64 568B ...
    ...            ...
    n2ovmr        (time) float64 5kB ...
    f11vmr        (time) float64 5kB ...
    f12vmr        (time) float64 5kB ...
    sol_tsi       (time) float64 5kB ...
    nsteph        (time) float64 5kB ...
    TREFHT        (time, lat, lon) float32 133MB ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BWmaHIST.f09_g17.release-cesm2.1.3.WACCM-MA-1deg...
    logname:           mmills
    host:              r6i6n31
    initial_file:      b.e21.BWma1850.f09_g17.release-cesm2.1.3.c20200918.cam...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

## Timeseries

In [10]:
# weights for latitudes
weights = np.cos(np.deg2rad(temp_yearly.lat))

# calculate preindustrial avg temp
pi_temp = temp_yearly.weighted(weights).mean(dim=('lat','lon')).sel(year=slice(1850,1880)).mean(dim="year")

# calculate temp anomaly
temp_anomaly = temp_yearly.weighted(weights).mean(dim=('lat', 'lon'))-pi_temp

# plot temp anomaly timeseries
temp_anomaly.hvplot.line(
    ylabel="Surface Temperature anomaly (K)"
)

:Curve   [year]   (TREFHT)

In [11]:
# plot the temp difference between preindustrial and 2000-2010 decade
pi_map = temp_yearly.sel(year=slice(1850,1880)).mean(dim="year")
chosen_decade = temp_yearly.sel(year=slice(2000,2010)).mean(dim="year")
(chosen_decade-pi_map).hvplot.image(
    x="lon", y="lat", cmap="coolwarm", clim=(-5, 5),
    geo=True,  # Enable geographic features
    coastline=True,  # Add coastlines
    features=['borders'] # Add country borders
)

:Overlay
   .Image.I     :Image   [lon,lat]   (TREFHT)
   .Coastline.I :Feature   [Longitude,Latitude]
   .Borders.I   :Feature   [Longitude,Latitude]

# Bonus content

In [12]:
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
hv.extension("bokeh") # or "plotly"

# --- pick your variable (first timestep if 'time' exists) ---
da = ds["TREFHT"]-273.15
if "time" in da.dims:
    da = da.isel(time=0)  # adjust if you want a different time slice

# --- flatten to a tidy DataFrame: columns ['lat', 'TREFHT'] (+others) ---
df = da.to_dataframe(name="TREFHT").reset_index()[["lat", "TREFHT"]].dropna()

# --- build latitude-band labels (N/S × 0–15, 15–30, 30–60, 60–90) ---
mag_labels = ["0–15", "15–30", "30–60", "60–90"]
mag_bins = pd.cut(np.abs(df["lat"]),
                  bins=[0, 15, 30, 60, 90],
                  labels=mag_labels,
                  include_lowest=True, right=True)

hemi = np.where(df["lat"] >= 0, "N", "S")
df["band"] = (pd.Series(hemi) + " " + mag_bins.astype(str) + "°")

# optional: sort legend in a nice order
order = ["S 60–90°","S 30–60°","S 15–30°","S 0–15°",
         "N 0–15°","N 15–30°","N 30–60°","N 60–90°"]
df["band"] = pd.Categorical(df["band"], categories=order, ordered=True)

# --- KDE by latitude band ---
kde_plot = df.hvplot.kde(
    "TREFHT",
    by="band",
    alpha=0.4,
    line_width=2,
    width=800,
    height=400,
    legend="right",
    title="TREFHT distribution by latitude band"
)
kde_plot

:NdOverlay   [band]
   :Distribution   [TREFHT]   (Density)